In [ ]:
!pip install transformers datasets

In [ ]:
from datasets import load_dataset

# Загружаем датасет CoNLL-2003
dataset = load_dataset("conll2003")

# Изучение структуры данных
print(dataset)

# Извлекаем тренировочные, валидационные и тестовые данные
train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']

# Пример данных
print(train_data[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})
{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}


In [ ]:
BERT требует специальной токенизации, которая учитывает подслова (subword tokenization).
Нам нужно будет токенизировать предложения с помощью токенизатора BERT, а затем адаптировать разметку сущностей к новой токенизации.

In [ ]:
from transformers import BertTokenizerFast

# Загрузка предобученного токенизатора BERT
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

# Функция для токенизации текста и адаптации меток NER
def tokenize_and_align_labels(examples):
    # Добавляем padding и truncation
    tokenized_inputs = tokenizer(
        examples['tokens'],
        truncation=True,
        is_split_into_words=True,
        padding='max_length',  # Добавляем padding до максимальной длины
        max_length=128  # Максимальная длина последовательности (можно варьировать)
    )

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Получаем индексы слов после токенизации
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:  # Пропускаем спецсимволы
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Новое слово
                label_ids.append(label[word_idx])
            else:  # Подслова
                label_ids.append(-100)  # Для подслов добавляем -100, чтобы их игнорировать при обучении
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Токенизация и выравнивание меток для всех сплитов
tokenized_train = train_data.map(tokenize_and_align_labels, batched=True)
tokenized_val = val_data.map(tokenize_and_align_labels, batched=True)
tokenized_test = test_data.map(tokenize_and_align_labels, batched=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
from transformers import BertForTokenClassification, TrainingArguments, Trainer

model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(dataset['train'].features['ner_tags'].feature.names))

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    # Добавляем padding и truncation
    gradient_accumulation_steps=2,  # Для более стабильного обучения
    fp16=True  # Использование 16-битных вычислений для ускорения
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,  # Указываем токенизатор
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.038700,0.044720
2,0.021200,0.039096
3,0.019100,0.037016


TrainOutput(global_step=1317, training_loss=0.07130146618358275, metrics={'train_runtime': 341.0475, 'train_samples_per_second': 123.511, 'train_steps_per_second': 3.862, 'total_flos': 2751824963545344.0, 'train_loss': 0.07130146618358275, 'epoch': 3.0})

In [ ]:
#Снижение Loss на валидации говорит о том, что модель не переобучается и продолжает хорошо обобщать на новых данных.

In [ ]:
import torch

# Проверяем, доступен ли GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Перемещаем модель на устройство
model.to(device)

def predict_ner_for_sentence(sentence):
    # Токенизация предложения
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)

    # Перемещаем входные данные на устройство (то же, что и модель)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Модель BERT делает предсказание для токенов
    outputs = model(**inputs)

    # Получаем предсказания (логиты) и преобразуем их в метки
    predictions = outputs.logits.argmax(dim=-1).squeeze().tolist()

    # Преобразуем токены и их предсказанные метки обратно в человекочитаемый формат
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'].squeeze().tolist())
    predicted_labels = [dataset['train'].features['ner_tags'].feature.names[pred] for pred in predictions]

    # Выводим токены вместе с предсказанными метками
    for token, label in zip(tokens, predicted_labels):
        print(f"{token}: {label}")

In [ ]:
sentence = "John Smith works at Google in New York"
predict_ner_for_sentence(sentence)

[CLS]: O
John: B-PER
Smith: I-PER
works: O
at: O
Google: B-ORG
in: O
New: B-LOC
York: I-LOC
[SEP]: O
